In [1]:
# print(os.getcwd())
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os, sys
from uncertainties import ufloat
# from numba import jit, njit, vectorize, prange
from importlib import reload
import py_library.my_plots_library as plib
import py_library.stopwatch as stopwatch
import py_library.simulate_lib as slib
import proposal as pp
from distributed import Client
from dask.distributed import performance_report
import dask.dataframe as dd
import dask.bag as db

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams.update({'figure.dpi':70})
# os.chdir(os.path.dirname(__file__))  # wichtig wenn nicht über ipython ausgeführt
client = Client("localhost:8786") # phobos
FLOAT_TYPE = np.float64
{
'''
1e7
workers:
2400 KilledWorker: ('finalize-6d57127f026613e3fce3aa1731d26349', <WorkerState 'tcp://127.0.0.1:43417', status: closed, memory: 0, processing: 1>) 
240 
24 2 worker sind abgeschmiert aber es lief durch 
'''
}
show_plots = True
print_results = False
silent = True
# hdf_folder = 'data_hdf/'
hdf_folder = '/scratch/mschoenfeld/data_hdf/'


file_name = "EcoMug_gaisser_30deg_1e7_min2e2_max2e5.hdf"


file_name = "EcoMug_gaisser_30deg_1e6_min4e2_max2e5.hdf"

file_name = "EcoMug_gaisser_30deg_3e7_min5e2_max2e5.hdf" # 1517.6 s
file_name = "EcoMug_gaisser_30deg_1e7_min5e2_max2e5.hdf" 

file_name = "EcoMug_gaisser_30deg_1e6_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e5_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf"
vcut = ''
multiple_scattering = 'HighlandIntegral'
multiple_scattering = 'Moliere'
vcut = 1
vcut = 0.01
vcut = 0.0001
vcut = 0.001
worker_number = 23
print(f'{file_name} | worker_number = {worker_number}')

# %
######################################################################
######################################################################
t1 = stopwatch.stopwatch(
    title='full proposal init and simulation parallized with dask',
    time_unit='s')
t1.task('initialize proposal, making interpol tables')

import d_pp_lib as proper
client.upload_file('d_pp_lib.py')
reload(proper)
pp_config = f'v{vcut}_{multiple_scattering}'
print(f'PROPOSAL config = {proper.config} | ppconfig = {pp_config}')
reload(stopwatch)
reload(plib)

STATISTICS = len(pd.read_hdf(
    hdf_folder+file_name, key=f'main', columns=['charge']))
chunksize = round((STATISTICS/worker_number))+1

meta={
    'hit_detector': bool, 
    'distances_f_raw': FLOAT_TYPE, 
    'energies_f_raw': FLOAT_TYPE, 
    'energies_i_raw': FLOAT_TYPE, 
    'point1x_raw': FLOAT_TYPE, 
    'point1y_raw': FLOAT_TYPE, 
    'point1z_raw': FLOAT_TYPE, 
    'point2x_raw': FLOAT_TYPE,
    'point2y_raw': FLOAT_TYPE,
    'point2z_raw': FLOAT_TYPE
}

t1.task('dask compute', True)
''' 
future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
    # own approach
    ddf = client.scatter(ddf)
    dfb = ddf.result().to_bag()
    # dfb = client.map(proper.pp_propagate, db.from_delayed(dfb) ) #27s
    # dfb = client.map(proper.pp_propagate, dfb) #27s
'''
with performance_report(filename="dask-report.html"):
    ddf = dd.read_hdf(hdf_folder+file_name, key='main',
                    columns=['energy', 'theta', 'phi', 'charge'], chunksize=chunksize)
    dfb = ddf.to_bag()

    dfb = dfb.map(proper.pp_propagate) #27s
    ddfr = dfb.to_dataframe(meta=meta)
    # ddfr.to_hdf(hdf_folder+'results_raw_'+file_name, key=f'main', format='table')
    results = client.compute(ddfr, pure=False).result()

t1.stop(silent)
# %
t2 = stopwatch.stopwatch(title='processing of results')
t2.task('nachbereitung')
######################################################################
######################################################################


hit_detector = np.array(results['hit_detector'], dtype=bool)
distances_f_raw = np.array(results['distances_f_raw'], dtype=FLOAT_TYPE)
energies_f_raw = np.array(results['energies_f_raw'], dtype=FLOAT_TYPE)
energies_i_raw = np.array(results['energies_i_raw'], dtype=FLOAT_TYPE)
point1x_raw = np.array(results['point1x_raw'], dtype=FLOAT_TYPE)
point1y_raw = np.array(results['point1y_raw'], dtype=FLOAT_TYPE)
point1z_raw = np.array(results['point1z_raw'], dtype=FLOAT_TYPE)
point2x_raw = np.array(results['point2x_raw'], dtype=FLOAT_TYPE)
point2y_raw = np.array(results['point2y_raw'], dtype=FLOAT_TYPE)
point2z_raw = np.array(results['point2z_raw'], dtype=FLOAT_TYPE)

counter = int(sum(hit_detector))  #len von allen die True sind
energies_f = np.zeros(counter, dtype=FLOAT_TYPE)
energies_i = np.zeros(counter, dtype=FLOAT_TYPE)
distances_f = np.zeros(counter, dtype=FLOAT_TYPE)
start_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
end_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
start_end_points = np.zeros(shape=(counter*2, 3), dtype=FLOAT_TYPE)

t2.task('1')
i2 = 0
for i in range(STATISTICS):
    if hit_detector[i] == True:
        energies_f[i2] = energies_f_raw[i]
        energies_i[i2] = energies_i_raw[i]
        distances_f[i2] = distances_f_raw[i]

        start_points[i2] = np.array([point1x_raw[i], point1y_raw[i], point1z_raw[i]])
        end_points[i2] = np.array([point2x_raw[i], point2y_raw[i], point2z_raw[i]])

        start_end_points[i2*2] = start_points[i2]
        start_end_points[i2*2+1] = end_points[i2]
        i2 += 1

t2.task('2')
df = pd.DataFrame()
df['energies_f'] = energies_f
df['energies_i'] = energies_i
df['distances'] = distances_f
df['point1x'] = start_points[:, 0]
df['point1y'] = start_points[:, 1]
df['point1z'] = start_points[:, 2]
df['point2x'] = end_points[:, 0]
df['point2y'] = end_points[:, 1]
df['point2z'] = end_points[:, 2]

t2.task('3')
t2.task('write to HDF file')
file_name_results = f'results_{pp_config}_{file_name}'
df.to_hdf(hdf_folder+file_name_results, key=f'main', format='table')
counter = ufloat(counter, counter*0.01)

EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf | worker_number = 23
PROPOSAL config = sandstein_det_genauer_Wasser.json | ppconfig = v0.001_Moliere
dask compute took  4016.2 s


In [16]:
counter_u = ufloat(counter, 1/np.sqrt(counter))
t2.task('4')
s1 = f'{counter_u:.1f} of {STATISTICS:.0e} ({counter_u/STATISTICS*100:.4}%) detector hits'
# s1 = f'{counter} of {STATISTICS:.0e} %) detector hits'
s2 = f'min(E_i) at detector = {min(energies_i)/1000:.1f} GeV'
print(f'{s1} | {s2}')

t2.stop(silent)

774878.0+/-0.0 of 1e+07 (7.749+/-0.000%) detector hits | min(E_i) at detector = 869.4 GeV


In [7]:
# print(os.getcwd())
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os, sys
from uncertainties import ufloat
# from numba import jit, njit, vectorize, prange
from importlib import reload
import py_library.my_plots_library as plib
import py_library.stopwatch as stopwatch
import py_library.simulate_lib as slib
import proposal as pp
from distributed import Client
from dask.distributed import performance_report
import dask.dataframe as dd
import dask.bag as db

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams.update({'figure.dpi':70})
# os.chdir(os.path.dirname(__file__))  # wichtig wenn nicht über ipython ausgeführt
client = Client("localhost:8786") # phobos
FLOAT_TYPE = np.float64
{
'''
1e7
workers:
2400 KilledWorker: ('finalize-6d57127f026613e3fce3aa1731d26349', <WorkerState 'tcp://127.0.0.1:43417', status: closed, memory: 0, processing: 1>) 
240 
24 2 worker sind abgeschmiert aber es lief durch 
'''
}
show_plots = True
print_results = False
silent = True
# hdf_folder = 'data_hdf/'
hdf_folder = '/scratch/mschoenfeld/data_hdf/'


file_name = "EcoMug_gaisser_30deg_1e7_min2e2_max2e5.hdf"


file_name = "EcoMug_gaisser_30deg_1e6_min4e2_max2e5.hdf"

file_name = "EcoMug_gaisser_30deg_3e7_min5e2_max2e5.hdf" # 1517.6 s
file_name = "EcoMug_gaisser_30deg_1e7_min5e2_max2e5.hdf" 

file_name = "EcoMug_gaisser_30deg_1e6_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e5_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf"
vcut = ''
multiple_scattering = 'HighlandIntegral'
multiple_scattering = 'Moliere'
vcut = 1
vcut = 0.01
vcut = 0.001
vcut = 0.0001
vcut = 0.0008
worker_number = 23
print(f'{file_name} | worker_number = {worker_number}')

# %
######################################################################
######################################################################
t1 = stopwatch.stopwatch(
    title='full proposal init and simulation parallized with dask',
    time_unit='s')
t1.task('initialize proposal, making interpol tables')

import d_pp_lib as proper
client.upload_file('d_pp_lib.py')
reload(proper)
pp_config = f'v{vcut}_{multiple_scattering}'
print(f'PROPOSAL config = {proper.config} | ppconfig = {pp_config}')
reload(stopwatch)
reload(plib)

STATISTICS = len(pd.read_hdf(
    hdf_folder+file_name, key=f'main', columns=['charge']))
chunksize = round((STATISTICS/worker_number))+1

meta={
    'hit_detector': bool, 
    'distances_f_raw': FLOAT_TYPE, 
    'energies_f_raw': FLOAT_TYPE, 
    'energies_i_raw': FLOAT_TYPE, 
    'point1x_raw': FLOAT_TYPE, 
    'point1y_raw': FLOAT_TYPE, 
    'point1z_raw': FLOAT_TYPE, 
    'point2x_raw': FLOAT_TYPE,
    'point2y_raw': FLOAT_TYPE,
    'point2z_raw': FLOAT_TYPE
}

t1.task('dask tasks', True)
''' 
future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
    # own approach
    ddf = client.scatter(ddf)
    dfb = ddf.result().to_bag()
    # dfb = client.map(proper.pp_propagate, db.from_delayed(dfb) ) #27s
    # dfb = client.map(proper.pp_propagate, dfb) #27s
'''
with performance_report(filename="dask-report.html"):
    ddf = dd.read_hdf(hdf_folder+file_name, key='main',
                    columns=['energy', 'theta', 'phi', 'charge'], chunksize=chunksize)
    dfb = ddf.to_bag()

    dfb = dfb.map(proper.pp_propagate) #27s
    ddfr = dfb.to_dataframe(meta=meta)
    # ddfr.to_hdf(hdf_folder+'results_raw_'+file_name, key=f'main', format='table')
    results = client.compute(ddfr, pure=False).result()

t1.stop(silent)
# %
t2 = stopwatch.stopwatch(title='processing of results')
t2.task('nachbereitung')
######################################################################
######################################################################


hit_detector = np.array(results['hit_detector'], dtype=bool)
distances_f_raw = np.array(results['distances_f_raw'], dtype=FLOAT_TYPE)
energies_f_raw = np.array(results['energies_f_raw'], dtype=FLOAT_TYPE)
energies_i_raw = np.array(results['energies_i_raw'], dtype=FLOAT_TYPE)
point1x_raw = np.array(results['point1x_raw'], dtype=FLOAT_TYPE)
point1y_raw = np.array(results['point1y_raw'], dtype=FLOAT_TYPE)
point1z_raw = np.array(results['point1z_raw'], dtype=FLOAT_TYPE)
point2x_raw = np.array(results['point2x_raw'], dtype=FLOAT_TYPE)
point2y_raw = np.array(results['point2y_raw'], dtype=FLOAT_TYPE)
point2z_raw = np.array(results['point2z_raw'], dtype=FLOAT_TYPE)

counter = int(sum(hit_detector))  #len von allen die True sind
energies_f = np.zeros(counter, dtype=FLOAT_TYPE)
energies_i = np.zeros(counter, dtype=FLOAT_TYPE)
distances_f = np.zeros(counter, dtype=FLOAT_TYPE)
start_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
end_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
start_end_points = np.zeros(shape=(counter*2, 3), dtype=FLOAT_TYPE)

t2.task('1')
i2 = 0
for i in range(STATISTICS):
    if hit_detector[i] == True:
        energies_f[i2] = energies_f_raw[i]
        energies_i[i2] = energies_i_raw[i]
        distances_f[i2] = distances_f_raw[i]

        start_points[i2] = np.array([point1x_raw[i], point1y_raw[i], point1z_raw[i]])
        end_points[i2] = np.array([point2x_raw[i], point2y_raw[i], point2z_raw[i]])

        start_end_points[i2*2] = start_points[i2]
        start_end_points[i2*2+1] = end_points[i2]
        i2 += 1

t2.task('2')
df = pd.DataFrame()
df['energies_f'] = energies_f
df['energies_i'] = energies_i
df['distances'] = distances_f
df['point1x'] = start_points[:, 0]
df['point1y'] = start_points[:, 1]
df['point1z'] = start_points[:, 2]
df['point2x'] = end_points[:, 0]
df['point2y'] = end_points[:, 1]
df['point2z'] = end_points[:, 2]

t2.task('3')
t2.task('write to HDF file')
file_name_results = f'results_{pp_config}_{file_name}'
df.to_hdf(hdf_folder+file_name_results, key=f'main', format='table')
counter_u = ufloat(counter, 1/np.sqrt(counter))
t2.task('4')
s1 = f'{counter_u:.1f} of {STATISTICS:.0e} ({counter_u/STATISTICS*100:.4}%) detector hits'
# s1 = f'{counter} of {STATISTICS:.0e} %) detector hits'
s2 = f'min(E_i) at detector = {min(energies_i)/1000:.1f} GeV'
print(f'{s1} | {s2}')

t2.stop(silent)

EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf | worker_number = 23
PROPOSAL config = sandstein_det_genauer_Wasser.json | ppconfig = v0.0008_Moliere
dask tasks took    15.1 s
766.0+/-0.0 of 1e+04 (7.660+/-0.000%) detector hits | min(E_i) at detector = 1018.2 GeV


In [8]:
# print(os.getcwd())
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os, sys
from uncertainties import ufloat
# from numba import jit, njit, vectorize, prange
from importlib import reload
import py_library.my_plots_library as plib
import py_library.stopwatch as stopwatch
import py_library.simulate_lib as slib
import proposal as pp
from distributed import Client
from dask.distributed import performance_report
import dask.dataframe as dd
import dask.bag as db

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams.update({'figure.dpi':70})
# os.chdir(os.path.dirname(__file__))  # wichtig wenn nicht über ipython ausgeführt
client = Client("localhost:8786") # phobos
FLOAT_TYPE = np.float64
{
'''
1e7
workers:
2400 KilledWorker: ('finalize-6d57127f026613e3fce3aa1731d26349', <WorkerState 'tcp://127.0.0.1:43417', status: closed, memory: 0, processing: 1>) 
240 
24 2 worker sind abgeschmiert aber es lief durch 
'''
}
show_plots = True
print_results = False
silent = True
# hdf_folder = 'data_hdf/'
hdf_folder = '/scratch/mschoenfeld/data_hdf/'


file_name = "EcoMug_gaisser_30deg_1e7_min2e2_max2e5.hdf"


file_name = "EcoMug_gaisser_30deg_1e6_min4e2_max2e5.hdf"

file_name = "EcoMug_gaisser_30deg_3e7_min5e2_max2e5.hdf" # 1517.6 s
file_name = "EcoMug_gaisser_30deg_1e7_min5e2_max2e5.hdf" 

file_name = "EcoMug_gaisser_30deg_1e6_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e5_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf"
vcut = ''
multiple_scattering = 'HighlandIntegral'
multiple_scattering = 'Moliere'
vcut = 1
vcut = 0.01
vcut = 0.001
vcut = 0.0001
vcut = 0.0007
worker_number = 23
print(f'{file_name} | worker_number = {worker_number}')

# %
######################################################################
######################################################################
t1 = stopwatch.stopwatch(
    title='full proposal init and simulation parallized with dask',
    time_unit='s')
t1.task('initialize proposal, making interpol tables')

import d_pp_lib as proper
client.upload_file('d_pp_lib.py')
reload(proper)
pp_config = f'v{vcut}_{multiple_scattering}'
print(f'PROPOSAL config = {proper.config} | ppconfig = {pp_config}')
reload(stopwatch)
reload(plib)

STATISTICS = len(pd.read_hdf(
    hdf_folder+file_name, key=f'main', columns=['charge']))
chunksize = round((STATISTICS/worker_number))+1

meta={
    'hit_detector': bool, 
    'distances_f_raw': FLOAT_TYPE, 
    'energies_f_raw': FLOAT_TYPE, 
    'energies_i_raw': FLOAT_TYPE, 
    'point1x_raw': FLOAT_TYPE, 
    'point1y_raw': FLOAT_TYPE, 
    'point1z_raw': FLOAT_TYPE, 
    'point2x_raw': FLOAT_TYPE,
    'point2y_raw': FLOAT_TYPE,
    'point2z_raw': FLOAT_TYPE
}

t1.task('dask tasks', True)
''' 
future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
    # own approach
    ddf = client.scatter(ddf)
    dfb = ddf.result().to_bag()
    # dfb = client.map(proper.pp_propagate, db.from_delayed(dfb) ) #27s
    # dfb = client.map(proper.pp_propagate, dfb) #27s
'''
with performance_report(filename="dask-report.html"):
    ddf = dd.read_hdf(hdf_folder+file_name, key='main',
                    columns=['energy', 'theta', 'phi', 'charge'], chunksize=chunksize)
    dfb = ddf.to_bag()

    dfb = dfb.map(proper.pp_propagate) #27s
    ddfr = dfb.to_dataframe(meta=meta)
    # ddfr.to_hdf(hdf_folder+'results_raw_'+file_name, key=f'main', format='table')
    results = client.compute(ddfr, pure=False).result()

t1.stop(silent)
# %
t2 = stopwatch.stopwatch(title='processing of results')
t2.task('nachbereitung')
######################################################################
######################################################################


hit_detector = np.array(results['hit_detector'], dtype=bool)
distances_f_raw = np.array(results['distances_f_raw'], dtype=FLOAT_TYPE)
energies_f_raw = np.array(results['energies_f_raw'], dtype=FLOAT_TYPE)
energies_i_raw = np.array(results['energies_i_raw'], dtype=FLOAT_TYPE)
point1x_raw = np.array(results['point1x_raw'], dtype=FLOAT_TYPE)
point1y_raw = np.array(results['point1y_raw'], dtype=FLOAT_TYPE)
point1z_raw = np.array(results['point1z_raw'], dtype=FLOAT_TYPE)
point2x_raw = np.array(results['point2x_raw'], dtype=FLOAT_TYPE)
point2y_raw = np.array(results['point2y_raw'], dtype=FLOAT_TYPE)
point2z_raw = np.array(results['point2z_raw'], dtype=FLOAT_TYPE)

counter = int(sum(hit_detector))  #len von allen die True sind
energies_f = np.zeros(counter, dtype=FLOAT_TYPE)
energies_i = np.zeros(counter, dtype=FLOAT_TYPE)
distances_f = np.zeros(counter, dtype=FLOAT_TYPE)
start_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
end_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
start_end_points = np.zeros(shape=(counter*2, 3), dtype=FLOAT_TYPE)

t2.task('1')
i2 = 0
for i in range(STATISTICS):
    if hit_detector[i] == True:
        energies_f[i2] = energies_f_raw[i]
        energies_i[i2] = energies_i_raw[i]
        distances_f[i2] = distances_f_raw[i]

        start_points[i2] = np.array([point1x_raw[i], point1y_raw[i], point1z_raw[i]])
        end_points[i2] = np.array([point2x_raw[i], point2y_raw[i], point2z_raw[i]])

        start_end_points[i2*2] = start_points[i2]
        start_end_points[i2*2+1] = end_points[i2]
        i2 += 1

t2.task('2')
df = pd.DataFrame()
df['energies_f'] = energies_f
df['energies_i'] = energies_i
df['distances'] = distances_f
df['point1x'] = start_points[:, 0]
df['point1y'] = start_points[:, 1]
df['point1z'] = start_points[:, 2]
df['point2x'] = end_points[:, 0]
df['point2y'] = end_points[:, 1]
df['point2z'] = end_points[:, 2]

t2.task('3')
t2.task('write to HDF file')
file_name_results = f'results_{pp_config}_{file_name}'
df.to_hdf(hdf_folder+file_name_results, key=f'main', format='table')
counter_u = ufloat(counter, 1/np.sqrt(counter))
t2.task('4')
s1 = f'{counter_u:.1f} of {STATISTICS:.0e} ({counter_u/STATISTICS*100:.4}%) detector hits'
# s1 = f'{counter} of {STATISTICS:.0e} %) detector hits'
s2 = f'min(E_i) at detector = {min(energies_i)/1000:.1f} GeV'
print(f'{s1} | {s2}')

t2.stop(silent)

EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf | worker_number = 23
PROPOSAL config = sandstein_det_genauer_Wasser.json | ppconfig = v0.0007_Moliere
dask tasks took    17.3 s
799.0+/-0.0 of 1e+04 (7.990+/-0.000%) detector hits | min(E_i) at detector = 1030.3 GeV


In [1]:
# print(os.getcwd())
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os, sys
from uncertainties import ufloat
# from numba import jit, njit, vectorize, prange
from importlib import reload
import py_library.my_plots_library as plib
import py_library.stopwatch as stopwatch
import py_library.simulate_lib as slib
import proposal as pp
from distributed import Client
from dask.distributed import performance_report
import dask.dataframe as dd
import dask.bag as db

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams.update({'figure.dpi':70})
# os.chdir(os.path.dirname(__file__))  # wichtig wenn nicht über ipython ausgeführt
client = Client("localhost:8786") # phobos
FLOAT_TYPE = np.float64
{
'''
1e7
workers:
2400 KilledWorker: ('finalize-6d57127f026613e3fce3aa1731d26349', <WorkerState 'tcp://127.0.0.1:43417', status: closed, memory: 0, processing: 1>) 
240 
24 2 worker sind abgeschmiert aber es lief durch 
'''
}
show_plots = True
print_results = False
silent = True
# hdf_folder = 'data_hdf/'
hdf_folder = '/scratch/mschoenfeld/data_hdf/'


file_name = "EcoMug_gaisser_30deg_1e7_min2e2_max2e5.hdf"


file_name = "EcoMug_gaisser_30deg_1e6_min4e2_max2e5.hdf"

file_name = "EcoMug_gaisser_30deg_3e7_min5e2_max2e5.hdf" # 1517.6 s
file_name = "EcoMug_gaisser_30deg_1e7_min5e2_max2e5.hdf" 

file_name = "EcoMug_gaisser_30deg_1e6_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e5_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf"
vcut = ''
multiple_scattering = 'HighlandIntegral'
multiple_scattering = 'Moliere'
vcut = 1
vcut = 0.01
vcut = 0.0001
vcut = 0.0008
worker_number = 23
print(f'{file_name} | worker_number = {worker_number}')

# %
######################################################################
######################################################################
t1 = stopwatch.stopwatch(
    title='full proposal init and simulation parallized with dask',
    time_unit='s')
t1.task('initialize proposal, making interpol tables')

import d_pp_lib as proper
client.upload_file('d_pp_lib.py')
reload(proper)
pp_config = f'v{vcut}_{multiple_scattering}'
print(f'PROPOSAL config = {proper.config} | ppconfig = {pp_config}')
reload(stopwatch)
reload(plib)

STATISTICS = len(pd.read_hdf(
    hdf_folder+file_name, key=f'main', columns=['charge']))
chunksize = round((STATISTICS/worker_number))+1

meta={
    'hit_detector': bool, 
    'distances_f_raw': FLOAT_TYPE, 
    'energies_f_raw': FLOAT_TYPE, 
    'energies_i_raw': FLOAT_TYPE, 
    'point1x_raw': FLOAT_TYPE, 
    'point1y_raw': FLOAT_TYPE, 
    'point1z_raw': FLOAT_TYPE, 
    'point2x_raw': FLOAT_TYPE,
    'point2y_raw': FLOAT_TYPE,
    'point2z_raw': FLOAT_TYPE
}

t1.task('dask tasks', True)
''' 
future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
    # own approach
    ddf = client.scatter(ddf)
    dfb = ddf.result().to_bag()
    # dfb = client.map(proper.pp_propagate, db.from_delayed(dfb) ) #27s
    # dfb = client.map(proper.pp_propagate, dfb) #27s
'''
with performance_report(filename="dask-report.html"):
    ddf = dd.read_hdf(hdf_folder+file_name, key='main',
                    columns=['energy', 'theta', 'phi', 'charge'], chunksize=chunksize)
    dfb = ddf.to_bag()

    dfb = dfb.map(proper.pp_propagate) #27s
    ddfr = dfb.to_dataframe(meta=meta)
    # ddfr.to_hdf(hdf_folder+'results_raw_'+file_name, key=f'main', format='table')
    results = client.compute(ddfr, pure=False).result()

t1.stop(silent)
# %
t2 = stopwatch.stopwatch(title='processing of results')
t2.task('nachbereitung')
######################################################################
######################################################################


hit_detector = np.array(results['hit_detector'], dtype=bool)
distances_f_raw = np.array(results['distances_f_raw'], dtype=FLOAT_TYPE)
energies_f_raw = np.array(results['energies_f_raw'], dtype=FLOAT_TYPE)
energies_i_raw = np.array(results['energies_i_raw'], dtype=FLOAT_TYPE)
point1x_raw = np.array(results['point1x_raw'], dtype=FLOAT_TYPE)
point1y_raw = np.array(results['point1y_raw'], dtype=FLOAT_TYPE)
point1z_raw = np.array(results['point1z_raw'], dtype=FLOAT_TYPE)
point2x_raw = np.array(results['point2x_raw'], dtype=FLOAT_TYPE)
point2y_raw = np.array(results['point2y_raw'], dtype=FLOAT_TYPE)
point2z_raw = np.array(results['point2z_raw'], dtype=FLOAT_TYPE)

counter = int(sum(hit_detector))  #len von allen die True sind
energies_f = np.zeros(counter, dtype=FLOAT_TYPE)
energies_i = np.zeros(counter, dtype=FLOAT_TYPE)
distances_f = np.zeros(counter, dtype=FLOAT_TYPE)
start_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
end_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
start_end_points = np.zeros(shape=(counter*2, 3), dtype=FLOAT_TYPE)

t2.task('1')
i2 = 0
for i in range(STATISTICS):
    if hit_detector[i] == True:
        energies_f[i2] = energies_f_raw[i]
        energies_i[i2] = energies_i_raw[i]
        distances_f[i2] = distances_f_raw[i]

        start_points[i2] = np.array([point1x_raw[i], point1y_raw[i], point1z_raw[i]])
        end_points[i2] = np.array([point2x_raw[i], point2y_raw[i], point2z_raw[i]])

        start_end_points[i2*2] = start_points[i2]
        start_end_points[i2*2+1] = end_points[i2]
        i2 += 1

t2.task('2')
df = pd.DataFrame()
df['energies_f'] = energies_f
df['energies_i'] = energies_i
df['distances'] = distances_f
df['point1x'] = start_points[:, 0]
df['point1y'] = start_points[:, 1]
df['point1z'] = start_points[:, 2]
df['point2x'] = end_points[:, 0]
df['point2y'] = end_points[:, 1]
df['point2z'] = end_points[:, 2]

t2.task('3')
t2.task('write to HDF file')
file_name_results = f'results_{pp_config}_{file_name}'
df.to_hdf(hdf_folder+file_name_results, key=f'main', format='table')
counter_u = ufloat(counter, 1/np.sqrt(counter))
t2.task('4')
s1 = f'{counter_u:.1f} of {STATISTICS:.0e} ({counter_u/STATISTICS*100:.4}%) detector hits'
# s1 = f'{counter} of {STATISTICS:.0e} %) detector hits'
s2 = f'min(E_i) at detector = {min(energies_i)/1000:.1f} GeV'
print(f'{s1} | {s2}')

t2.stop(silent)

EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf | worker_number = 23
PROPOSAL config = sandstein_det_genauer_Wasser.json | ppconfig = v0.0008_Moliere
dask tasks took  4993.3 s
769205.0+/-0.0 of 1e+07 (7.692+/-0.000%) detector hits | min(E_i) at detector = 885.4 GeV


In [3]:
# print(os.getcwd())
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os, sys
from uncertainties import ufloat
# from numba import jit, njit, vectorize, prange
from importlib import reload
import py_library.my_plots_library as plib
import py_library.stopwatch as stopwatch
import py_library.simulate_lib as slib
import proposal as pp
from distributed import Client
from dask.distributed import performance_report
import dask.dataframe as dd
import dask.bag as db

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams.update({'figure.dpi':70})
# os.chdir(os.path.dirname(__file__))  # wichtig wenn nicht über ipython ausgeführt
client = Client("localhost:8786") # phobos
FLOAT_TYPE = np.float64
{
'''
1e7
workers:
2400 KilledWorker: ('finalize-6d57127f026613e3fce3aa1731d26349', <WorkerState 'tcp://127.0.0.1:43417', status: closed, memory: 0, processing: 1>) 
240 
24 2 worker sind abgeschmiert aber es lief durch 
'''
}
show_plots = True
print_results = False
silent = True
# hdf_folder = 'data_hdf/'
hdf_folder = '/scratch/mschoenfeld/data_hdf/'


file_name = "EcoMug_gaisser_30deg_1e7_min2e2_max2e5.hdf"


file_name = "EcoMug_gaisser_30deg_1e6_min4e2_max2e5.hdf"

file_name = "EcoMug_gaisser_30deg_3e7_min5e2_max2e5.hdf" # 1517.6 s
file_name = "EcoMug_gaisser_30deg_1e7_min5e2_max2e5.hdf" 

file_name = "EcoMug_gaisser_30deg_1e6_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e5_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e4_min6e2_max2e5.hdf"
file_name = "EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf"
vcut = ''
multiple_scattering = 'HighlandIntegral'
multiple_scattering = 'Moliere'
vcut = 1
vcut = 0.01
vcut = 0.0001
vcut = 0.0008
N_tasks = 230
print(f'{file_name} | N_tasks = {N_tasks}')

# %
######################################################################
######################################################################
t1 = stopwatch.stopwatch(
    title='full proposal init and simulation parallized with dask',
    time_unit='s')
t1.task('initialize proposal, making interpol tables')

import d_pp_lib as proper
client.upload_file('d_pp_lib.py')
reload(proper)
pp_config = f'v{vcut}_{multiple_scattering}'
print(f'PROPOSAL config = {proper.config} | ppconfig = {pp_config}')
reload(stopwatch)
reload(plib)

STATISTICS = len(pd.read_hdf(
    hdf_folder+file_name, key=f'main', columns=['charge']))
chunksize = round((STATISTICS/N_tasks))+1

meta={
    'hit_detector': bool, 
    'distances_f_raw': FLOAT_TYPE, 
    'energies_f_raw': FLOAT_TYPE, 
    'energies_i_raw': FLOAT_TYPE, 
    'point1x_raw': FLOAT_TYPE, 
    'point1y_raw': FLOAT_TYPE, 
    'point1z_raw': FLOAT_TYPE, 
    'point2x_raw': FLOAT_TYPE,
    'point2y_raw': FLOAT_TYPE,
    'point2z_raw': FLOAT_TYPE
}

t1.task('dask tasks', True)
''' 
future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
    # own approach
    ddf = client.scatter(ddf)
    dfb = ddf.result().to_bag()
    # dfb = client.map(proper.pp_propagate, db.from_delayed(dfb) ) #27s
    # dfb = client.map(proper.pp_propagate, dfb) #27s
'''
with performance_report(filename="dask-report.html"):
    ddf = dd.read_hdf(hdf_folder+file_name, key='main',
                    columns=['energy', 'theta', 'phi', 'charge'], chunksize=chunksize)
    dfb = ddf.to_bag()

    dfb = dfb.map(proper.pp_propagate) #27s
    ddfr = dfb.to_dataframe(meta=meta)
    # ddfr.to_hdf(hdf_folder+'results_raw_'+file_name, key=f'main', format='table')
    results = client.compute(ddfr, pure=False).result()

t1.stop(silent)
# %
t2 = stopwatch.stopwatch(title='processing of results')
t2.task('nachbereitung')
######################################################################
######################################################################


hit_detector = np.array(results['hit_detector'], dtype=bool)
distances_f_raw = np.array(results['distances_f_raw'], dtype=FLOAT_TYPE)
energies_f_raw = np.array(results['energies_f_raw'], dtype=FLOAT_TYPE)
energies_i_raw = np.array(results['energies_i_raw'], dtype=FLOAT_TYPE)
point1x_raw = np.array(results['point1x_raw'], dtype=FLOAT_TYPE)
point1y_raw = np.array(results['point1y_raw'], dtype=FLOAT_TYPE)
point1z_raw = np.array(results['point1z_raw'], dtype=FLOAT_TYPE)
point2x_raw = np.array(results['point2x_raw'], dtype=FLOAT_TYPE)
point2y_raw = np.array(results['point2y_raw'], dtype=FLOAT_TYPE)
point2z_raw = np.array(results['point2z_raw'], dtype=FLOAT_TYPE)

counter = int(sum(hit_detector))  #len von allen die True sind
energies_f = np.zeros(counter, dtype=FLOAT_TYPE)
energies_i = np.zeros(counter, dtype=FLOAT_TYPE)
distances_f = np.zeros(counter, dtype=FLOAT_TYPE)
start_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
end_points = np.zeros(shape=(counter, 3), dtype=FLOAT_TYPE)
start_end_points = np.zeros(shape=(counter*2, 3), dtype=FLOAT_TYPE)

t2.task('1')
i2 = 0
for i in range(STATISTICS):
    if hit_detector[i] == True:
        energies_f[i2] = energies_f_raw[i]
        energies_i[i2] = energies_i_raw[i]
        distances_f[i2] = distances_f_raw[i]

        start_points[i2] = np.array([point1x_raw[i], point1y_raw[i], point1z_raw[i]])
        end_points[i2] = np.array([point2x_raw[i], point2y_raw[i], point2z_raw[i]])

        start_end_points[i2*2] = start_points[i2]
        start_end_points[i2*2+1] = end_points[i2]
        i2 += 1

t2.task('2')
df = pd.DataFrame()
df['energies_f'] = energies_f
df['energies_i'] = energies_i
df['distances'] = distances_f
df['point1x'] = start_points[:, 0]
df['point1y'] = start_points[:, 1]
df['point1z'] = start_points[:, 2]
df['point2x'] = end_points[:, 0]
df['point2y'] = end_points[:, 1]
df['point2z'] = end_points[:, 2]

t2.task('3')
t2.task('write to HDF file')
file_name_results = f'results_{pp_config}_{file_name}'
df.to_hdf(hdf_folder+file_name_results, key=f'main', format='table')
counter_u = ufloat(counter, 1/np.sqrt(counter))
t2.task('4')
s1 = f'{counter_u:.1f} of {STATISTICS:.0e} ({counter_u/STATISTICS*100:.4}%) detector hits'
# s1 = f'{counter} of {STATISTICS:.0e} %) detector hits'
s2 = f'min(E_i) at detector = {min(energies_i)/1000:.1f} GeV'
print(f'{s1} | {s2}')

t2.stop(silent)

EcoMug_gaisser_30deg_1e7_min6e2_max2e5.hdf | N_tasks = 230
PROPOSAL config = sandstein_det_genauer_Wasser.json | ppconfig = v0.0008_Moliere
dask tasks took  4962.3 s
769323.0+/-0.0 of 1e+07 (7.693+/-0.000%) detector hits | min(E_i) at detector = 895.5 GeV
